In [1]:
import torch
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import os
from ultralytics import YOLO

In [2]:
torch.cuda.is_available()

True

In [3]:
model = YOLO('./YOLOV8-anchor-colab/runs/detect/train/weights/best.pt')  # load a custom model

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
def ret_landmark(img):
    with mp_holistic.Holistic(model_complexity = 0,
                          min_detection_confidence = 0.6,
                          min_tracking_confidence = 0.6,
                          refine_face_landmarks = True,
                          smooth_landmarks = True) as holistic:
        image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # cv2_imshow(image)
    face_arr = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros(468*3)
    return face_arr
    # print(results.face_landmarks)

In [ ]:
def gen_ldms_data(video_path,threshold=0.76):
    cap = cv2.VideoCapture(video_path)
    face_seq = []
    frame_count = 0;
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        results = model(frame)[0]

        for result in results.boxes.data.tolist():
            if result[4] > threshold:
                face_arr = ret_landmark(frame)
                face_seq.append(face_arr)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    face_seq = np.array(face_seq)
    face_seq.shape
    np.save(f'output/mesh_sequence_{video_path}.npy', face_seq)
    return face_seq


In [ ]:
video_path = 'yesma path rakha'
mesh_arr = gen_ldms_data(video_path=video_path, threshold=0.78)

In [4]:
VIDEOS_DIR = os.path.join('.', 'videos')

video_path = "D:/AI-Anchor/videodataset/English.mp4"
video_path_out = '{}_out.mp4'.format(video_path)

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))


threshold = 0.74

while ret:

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 anchor, 46.9ms
Speed: 6.5ms preprocess, 46.9ms inference, 172.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 43.8ms
Speed: 15.6ms preprocess, 43.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 46.9ms
Speed: 0.0ms preprocess, 46.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 47.3ms
Speed: 0.0ms preprocess, 47.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 31.3ms
Speed: 0.0ms preprocess, 31.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 7.0ms
Speed: 0.0ms preprocess, 7.0ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 